In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import data
df = pd.read_csv('data.csv')

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# remove latitude and longitude columns
df.drop(['Latitude'], axis=1, inplace=True)
df.drop(['Longitude'], axis=1, inplace=True)

In [ ]:
# find the number of missing values in each column
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# remove the rows with missing values in the column 'Daily tests'
df = df.dropna(subset=['Daily tests']).reset_index(drop = True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Removing countries with not enough data

In [ ]:
# find the sum of unique days in dataset
# total days between 1/1/2020 and 28/2/2021
total_days = 425 
total_countries = len(df['Entity'].unique())
# for each country, find the number of days 
countries = df['Entity'].unique()
for country in countries:
    # find the data frame for each country
    country_df = df[df['Entity'] == country]
    # find the number of days for each country
    country_days = country_df['Date'].nunique()
    # find the percent of days for each country
    percent = country_days / total_days
    if (percent<0.15): 
        # these countries have less than 15% of the days in the dataset
        # so we will remove them
        df.drop(df[df['Entity'] == country].index, inplace = True)
        # print(country, round(percent, 2))
# fix the index
df = df.reset_index(drop = True)
# print how many countries are left
print('Number of countries left: ',total_countries-len(df['Entity'].unique()))

Replace NaN values before first deaths and cases for each country with 0

In [ ]:
entityGroup = df.groupby('Entity')

for entity, group in entityGroup:
    for index in range(group.index[0], group.index[-1] + 1):
        if (df.isnull().iloc[index]['Cases']):
            df.at[index, 'Cases'] = 0
        else:
            break

for entity, group in entityGroup:
    for index in range(group.index[0], group.index[-1] + 1):
        if (df.isnull().iloc[index]['Deaths']):
            df.at[index, 'Deaths'] = 0
        else:
            break

df.isnull().sum().sort_values(ascending=False)

Convert total number of deaths and cases to daily number

In [ ]:
# add a new column to the DataFrame for cases today
df['cases_today'] = 0
# add a new column to the DataFrame for deaths today
df['deaths_today'] = 0

dfCopy = df.copy()
for entity, group in entityGroup:
    for index in range(group.index[0]+1, group.index[-1] + 1):
        df.at[index, 'cases_today'] = dfCopy.at[index, 'Cases'] - dfCopy.at[index - 1, 'Cases'] 
        df.at[index, 'deaths_today'] = dfCopy.at[index, 'Deaths'] - dfCopy.at[index - 1, 'Deaths']

Create Seasons Column\
1 (winter) -> December - February \
2 (spring) -> March - May \
3 (summer) -> June - August \
4 (autumn) -> September - November

In [ ]:
seasons = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'}

df['Date'] =  pd.to_datetime(df['Date'])
df['Season'] = df['Date'].apply(lambda x: seasons[(x.month - 1) // 3 + 1])

Correlation Heatmap

One hot encoding for seasons

In [ ]:
# df = pd.get_dummies(df, columns=['Season'])
# # remove winter column
# df.drop(['Season_Winter'], axis=1, inplace=True)

One hot encode Continent column

In [ ]:
# df = pd.get_dummies(df, columns=['Continent'])
# # remove Continent_Asia
# df.drop(['Continent_Asia'], axis=1, inplace=True)
df.head()

Extract a csv file with the cleaned data

In [ ]:
# export datta to csv
df.to_csv('data_cleaned.csv', index=False)